In [1]:
import os
import shutil
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import requests
import pandas as pd

In [2]:
def generate_url(isolate_id, database):
    url = f'http://rest.pubmlst.org/db/pubmlst_{database}_isolates/isolates/{isolate_id}/contigs_fasta'
    return url

def download_contigs(url, outfile):
    r = requests.get(url, params={'header': 'original_designation'})
    if r.status_code is 200:
        with open(outfile, 'w') as handle:
            handle.write(r.text)

In [3]:
filepath = '/media/NGS/SRA_2/pubMLST_Neisseria/BIGSdb_179004_1119847573_26598.xlsx'
isolate_info = pd.read_excel(filepath, dtype=object)

In [10]:
isolate_info = isolate_info.dropna(axis=1, how='all')
isolate_info = isolate_info[isolate_info['species'] == 'Neisseria meningitidis']
# isolate_info = isolate_info[isolate_info['clonal_complex (MLST)'] == 'ST-32 complex']
isolate_info.shape

(25862, 63)

In [11]:
dirpath = Path('/media/NGS/SRA_2/pubMLST_Neisseria/Contigs')
downloaded_ids = {int(filepath.stem) for filepath in dirpath.iterdir()}

In [12]:
ids = set(isolate_info['id']) - downloaded_ids

In [13]:
len(ids)

21944

In [14]:
database = 'neisseria'
outpath = Path('/media/NGS/SRA_2/pubMLST_Neisseria/Contigs')

In [15]:
with ThreadPoolExecutor(4) as executor:
    for _id in ids:
        url = generate_url(_id, database)
        outfile = outpath/f"{_id}.fas"
        executor.submit(download_contigs, url, outfile)